In [29]:
import untangle 
from glob import iglob as glob
import wordcount
from collections import Counter
from string import punctuation
from joblib import Parallel, delayed

In [7]:
LIWC_dict = "../LIWC2007/Dictionaries/LIWC2007_English131104.dic"

In [8]:
(categories, words) = wordcount.readDictFile(LIWC_dict)

Error in dict:  kind	<of>131/125	<of>135/126
Error in dict:  like	(02 134)125/464	(02 134)126	253


In [2]:
def read(filename):
    utterances = []
    f = untangle.parse(filename)

    for diag in f.movie.dialogue:
        utterances.extend(list(zip(map(lambda x: x.cdata, diag.speaker), map(lambda x: x.cdata, diag.utterance))))

    return utterances

In [25]:
def mapper(line):
    return (Counter([categories[y] for x in [words[_] for t in wordcount.preprocess(line).split(" ") for _ in words.keys() if _.match(t)] for y in x]),
            len(list(filter(lambda t: t not in punctuation, wordcount.preprocess(line).split(" ")))),\
            1.0 if len(line.strip()) > 0 else 0.0)

In [44]:
def process(name, movielines, cats = []):
    (counters, llens, lines) = zip(*Parallel(n_jobs = 4, verbose = 2)(delayed(mapper)(line[1]) for line in movielines))
    WC = wordcount.reducer(counters)
    num_words = sum(llens)
#     wps = num_words

    res = []
    for c in cats:
        res.append((c, round(100.0*WC[c]/num_words, 3)))
    
    return (name, num_words, res)

In [45]:
process("x_men", movielines, cats = ["i", "relig", "pronoun"])

[Parallel(n_jobs=4)]: Done 363 out of 363 | elapsed:    2.1s finished


('x_men', 3928, [('i', 6.39), ('relig', 0.509), ('pronoun', 23.32)])

H0: Use of pronouns differ across sexes

In [41]:
cats = ["pronoun"]